In [2]:
from modules.data_loading import load_train_data, load_test_data
import pandas as pd
import numpy as np
import ydata_profiling

In [6]:
df=pd.read_csv('train.csv', index_col=None)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv('train.csv', index=False)

In [3]:
null_cols = df.columns[df.isnull().any()]
null_cols

Index(['Dimensions du bâtiment', 'Éval. terrain', 'Superficie habitable',
       'Éval. bâtiment', 'Dimensions du terrain', 'Superficie du terrain',
       'Plan d'eau', 'Piscine', 'Stationnement (total)', 'Chauffage',
       'Eau (accès)', 'Foyers-Poêles', 'Équip./Serv.', 'Rénovations',
       'Inclusions', 'Exclusions', 'Remarques - Courtier', 'Addenda',
       'Nbre pièces', 'Nbre chambres (hors-sol + sous-sol)',
       'Nbre salles de bains + salles d'eau', 'Source'],
      dtype='object')

In [4]:
profile = df.profile_report()
profile.to_file("eda_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]